
<p align="center">
    <img src="https://github.com/GeostatsGuy/GeostatsPy/blob/master/TCG_color_logo.png?raw=true" width="220" height="240" />

</p>

### Interactive Spatial Bootstrap


#### Michael Pyrcz, Professor, The University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

This is a tutorial for / demonstration of **Bootstrap**. 

**YouTube Lecture**: check out my lecture on [Bootstrap](https://youtu.be/wCgdoImlLY0?si=lpTWz2H7QTdxHBy9). For your convenience here's a summary of salient points.


#### Bootstrap

Uncertainty in the sample statistics
* one source of uncertainty is the paucity of data.
* do 200 or even less wells provide a precise (and accurate estimate) of the mean? standard deviation? skew? P13?

Would it be useful to know the uncertainty in these statistics due to limited sampling?
* what is the impact of uncertainty in the mean porosity e.g. 20%+/-2%?

**Bootstrap** is a method to assess the uncertainty in a sample statistic by repeated random sampling with replacement.

Assumptions
* sufficient, representative sampling, identical, idependent samples

Limitations
1. assumes the samples are representative 
2. assumes stationarity
3. only accounts for uncertainty due to too few samples, e.g. no uncertainty due to changes away from data
4. does not account for boundary of area of interest 
5. assumes the samples are independent
6. does not account for other local information sources

The Bootstrap Approach (Efron, 1982)

Statistical resampling procedure to calculate uncertainty in a calculated statistic from the data itself.
* Does this work?  Prove it to yourself, for uncertainty in the mean solution is standard error: 

\begin{equation}
\sigma^2_\overline{x} = \frac{\sigma^2_s}{n}
\end{equation}

Extremely powerful - could calculate uncertainty in any statistic!  e.g. P13, skew etc.
* Would not be possible access general uncertainty in any statistic without bootstrap.
* Advanced forms account for spatial information and sampling strategy (game theory and Journel’s spatial bootstrap (1993).

Steps: 

1. assemble a sample set, must be representative, reasonable to assume independence between samples

2. optional: build a cumulative distribution function (CDF)
    * may account for declustering weights, tail extrapolation
    * could use analogous data to support

3. For $\ell = 1, \ldots, L$ realizations, do the following:

    * For $i = \alpha, \ldots, n$ data, do the following:

        * Draw a random sample with replacement from the sample set or Monte Carlo simulate from the CDF (if available). 

6. Calculate a realization of the sammary statistic of interest from the $n$ samples, e.g. $m^\ell$, $\sigma^2_{\ell}$. Return to 3 for another realization.

7. Compile and summarize the $L$ realizations of the statistic of interest.

#### Spatial Bootstrap

Journel (1993) developed methods for spatial bootstrap based on bootstrap resampling accounting for the locations of the data and the spatial continuity model.  

* One method to perform spatial bootstrap for uncertainty of a statistic is to adjust the number of data, $n$, to the **number of effective data**, then use the number of effective data instead of number of data of resamples with replacement for each bootstrap realization.

This number of effectve data may be calculated by:

* building multiple unconditional simulated realizations at the data locations only 

* calculating the variance of the average of each simulated realization

* then calculate the number effective data by manipulating the standard error in the average equation:

\begin{equation}
\sigma^2_\overline{x} = \frac{\sigma^2}{n}
\end{equation}

to be expressed as:

\begin{equation}
n^{'} = \frac{\sigma^2}{\sigma^2_\overline{x}}
\end{equation}

where $\sigma^2_\overline{x}$ is the variance of the average over bootstrap realizations $\ell = 1,\ldots,L$, $\sigma^2$ is the variance / sill of the problem.

#### Simulating Only at the Data Locations

We ultilize LU Simulation to efficiently calculate Gaussian realizations only at the data locations, based on the Lower-Upper Decomposition of the left-hand / redudancy martix of the simple kriging.  

* We multiple the Lower matrix ($n \times n$) with a random Gaussian vector ($1 \times n$) to calculate each realization at the data locations.

* We then take the average of each realization and then calculate the variance of the average over enough realizations.



This is a very powerful method, but it ignores the spatial context!

* here I provide demonstrations for spatial bootstrap, a variant of bootstrap that accounts for the spatial context.

#### Load the required libraries

The following code loads the required libraries. 

In [1]:
import geostatspy.GSLIB as GSLIB                              # GSLIB utilies, visualization and wrapper
import geostatspy.geostats as geostats                        # GSLIB methods convert to Python      
import geostatspy
print('GeostatsPy version: ' + str(geostatspy.__version__))

GeostatsPy version: 0.0.72


We will also need some standard packages. These should have been installed with Anaconda 3.

In [2]:
supress_warnings = True                                       # supress warnings?
ignore_warnings = True                                        # ignore warnings?
import numpy as np                                            # ndarrys for gridded data
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt                               # for plotting
import statsmodels.api as sm                                  # make PDFs
import math
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator) # control of axes ticks
import scipy as sp                                            # lower upper matrix decomposition
from scipy.interpolate import make_interp_spline              # smooth curves
import statsmodels.formula.api as smf    
from scipy.stats import norm                                  # Gaussian distribution PDF and sampling
import geostatspy.GSLIB as GSLIB    
import geostatspy.geostats as geostats    
import scipy.signal as signal                                 # kernel for convolution
from scipy import ndimage                               
from scipy import stats    
from ipywidgets import interactive                            # widgets and interactivity
from ipywidgets import widgets                                
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox
plt.rc('axes', axisbelow=True)                            # grid behind plotting elements
if supress_warnings == True:
    import warnings                                       # supress any warnings for this demonstration
    warnings.filterwarnings('ignore')                  
cmap = plt.cm.inferno                                     # default color bar, no bias and friendly for color vision defeciency

If you get a package import error, you may have to first install some of these packages. This can usually be accomplished by opening up a command window on Windows and then typing 'python -m pip install [package-name]'. More assistance is available with the respective package docs.  

#### Declare Functions

I just added a convenience functions to:

1. add major and minor gridlines
2. calculate a isotropic variogram between 2 points
3. calculate the azimuth between 2 points to rotate a label

In [3]:
def add_grid():
    plt.gca().grid(True, which='major',linewidth = 1.0); plt.gca().grid(True, which='minor',linewidth = 0.2) # add y grids
    plt.gca().tick_params(which='major',length=7); plt.gca().tick_params(which='minor', length=4)
    plt.gca().xaxis.set_minor_locator(AutoMinorLocator()); plt.gca().yaxis.set_minor_locator(AutoMinorLocator()) # turn on minor ticks     

def n_effective(df,xcol,ycol,seed,nreal,vario):
    """Calculate the number of effective data from spatial locations and spatial continuity model
    Used in bootstrap to account for spatial continuity, use n effective instead of number of data
    :param df: source DataFrame
    :param xcol: column with the X locations
    :param ycol: column with the Y locations
    :param seed: random number seed for the random sampling
    :param nreal: number of realizations to sample the variance of the average 
    :param vario: variogram model as a dictionary, see the GeostatsPy Package's GSLIB.make_variogram() function 
    :return: n_eff as effective number of data
    """ 

# Set constants
    np.random.seed(seed)
    PMX = 9999.9
    
# load the variogram
    nst = vario['nst']
    cc = np.zeros(nst); aa = np.zeros(nst); it = np.zeros(nst)
    ang = np.zeros(nst); anis = np.zeros(nst)
    c0 = vario['nug']; 
    cc[0] = vario['cc1']; it[0] = vario['it1']; ang[0] = vario['azi1']; 
    aa[0] = vario['hmaj1']; anis[0] = vario['hmin1']/vario['hmaj1'];
    if nst == 2:                                   # include 2nd structure if present (optional)
        cc[1] = vario['cc2']; it[1] = vario['it2']; ang[1] = vario['azi2']; 
        aa[1] = vario['hmaj2']; anis[1] = vario['hmin2']/vario['hmaj2'];
    
# Set up the rotation matrix
    rotmat, maxcov = geostats.setup_rotmat(c0,nst,it,cc,ang,PMX)
    
# Load the data
    nd = len(df)
    x = df[xcol].values
    y = df[ycol].values
    
# Calculate Symmetric Covariance Array - assuming variogram with spherical structure with range specified
    cov = np.zeros((nd,nd))
    var_range = 100.0
    for i in range(0, nd):
        x1 = x[i]; y1 = y[i]
        for j in range(0, nd):
            x2 = x[j]; y2 = y[j]
            cova = geostats.cova2(x1, y1, x2, y2, nst, c0, PMX, cc, aa, it, ang, anis, rotmat, maxcov)
            cov[i,j] = cova
            
# Lower and upper deconvolution            
    P, L, U = sp.linalg.lu(cov) 
    
# Build realization and calculate the average    
    realizations = np.zeros((nreal,nd))
    average_array = np.zeros(nreal)
    rand = np.zeros((nd)) 
    for l in range(0, nreal):
        rand = np.random.normal(loc = 0.0, scale = 1.0, size = nd)
        realizations[l,:] = np.matmul(L,rand)
        average_array[l] = np.average(realizations[l])
        
# Back out the number of effecitve data useing the standard error in the average
    var_average = np.var(average_array)
    n_eff = max(min(1.0/var_average, nd),1.0)    # filter n effective less than 1.0 or greater than number of data
    return n_eff, realizations
    
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

#### n Effective Demonstration

This dashboard visualizes the impact of spatial continuity range and number of samples on the effective number of data. 

* How many independent peices of information do you have given spatial correlation between the samples.

In [6]:
max_L = 100                                                   # maximum number of realizations to precalculate

l = widgets.Text(value=r'                                                      Number of Effective Data, Michael Pyrcz, Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

n = widgets.IntSlider(min=1, max = max_L, value = 1,step = 1,description = 'n',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
n.style.handle_color = 'gray'

vrange = widgets.FloatSlider(min=0.1, max = 200.0, value = 10.0,step=5,description = 'range',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
vrange.style.handle_color = 'gray'

seed = widgets.IntSlider(min=100, max = 999, value = 1, description = 's',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
seed.style.handle_color = 'gray'

window = widgets.Checkbox(value=False,description='Conv. Fit',disabled=False,layout=Layout(width='200px', height='30px'))
poly = widgets.Checkbox(value=True,description='Poly. Fit',disabled=False,layout=Layout(width='200px', height='30px'))

ui1 = widgets.HBox([n,vrange,seed,],kwargs = {'justify_content':'center'})
ui2 = widgets.HBox([window,poly,],kwargs = {'justify_content':'center'})
ui_all = widgets.VBox([l,ui1,ui2],)

def f_make1(n,vrange,seed,window,poly): 
    np.random.seed(seed=seed)
    values = np.random.rand(max_L*2)*100
    X,Y = np.split(values,2)
    df = pd.DataFrame(np.vstack((X[:n],Y[:n])).T,columns = ['X','Y'])
    
    ax1 = plt.subplot(121)
    plt.scatter(df['X'],df['Y'],color='red',edgecolor='black',zorder=10,label = 'Spatial Data')
    plt.gca().add_patch(plt.Rectangle((0, 0), 100, 100, fill=False,edgecolor='black',lw=2))
    
    for ipts in range(0,n):
        if ipts == 0:
            circle1 = plt.Circle((df['X'][ipts],df['Y'][ipts]),vrange,fill=True,color='red',edgecolor=None,
                lw=0.0,alpha=0.1,zorder=1,label='Spatial Continuity')
        else:
            circle1 = plt.Circle((df['X'][ipts],df['Y'][ipts]),vrange,fill=True,color='red',edgecolor=None,
                lw=0.0,alpha=0.1,zorder=1)
        plt.gca().add_patch(circle1)
    
    plt.xlim([-10,110]); plt.ylim([-10,110]); add_grid(); plt.xlabel("X (m)"); plt.ylabel("Y (m)") 
    plt.title('Random Spatial Dataset with Spatial Correlation Range Indicated')
    
    vario = GSLIB.make_variogram(nug=0.0,nst=1,it1=1,cc1=1.0,azi1=0.0,hmaj1=vrange,hmin1=vrange)
    neffective = n_effective(df,'X','Y',seed=seed,nreal=1000,vario=vario)[0]
    
    neffect_array = []; l_array = []
    for l in np.arange(1,max_L,2):
        l_array.append(l)
        df = pd.DataFrame(np.vstack((X[:l],Y[:l])).T,columns = ['X','Y'])
        neffect_array.append(n_effective(df,'X','Y',seed=seed,nreal=1000,vario=vario)[0])
    
    df_effective = pd.DataFrame(np.vstack((l_array,neffect_array)).T,columns = ['l','e'])    
    olsres2 = smf.ols(formula = 'e ~ l + I(l**2)+ I(l**3)-1', data = df_effective).fit()    
      
    plt.annotate(r'$n$ = ' + str(np.round(n,2)),(0,-5))
    plt.annotate(r'$n_{effective}$ = ' + str(np.round(neffective,2)),(15,-5))
    neffective_model = olsres2.params[2]*np.power(n,3) + olsres2.params[1]*np.power(n,2) + olsres2.params[0]*n
    plt.annotate(r'$\hat{n}_{effective}$ = ' + str(np.round(neffective_model,2)),(42,-5))
    plt.legend(loc = 'upper left')
        
    ax2 = plt.subplot(122)
    plt.plot([1,max_L],[1,max_L],color='black')
    plt.scatter(n,neffective,c='red',edgecolor='black',s=20,zorder=10)
    plt.scatter(n,neffective_model,c='darkorange',edgecolor='black',s=20,zorder=10)
    plt.scatter(l_array,neffect_array,c='grey',edgecolor='grey',lw=0,s=30,alpha=0.4,zorder=5)
    kernel = np.array([0.076923,0.230769,0.384615,0.230769,0.076923])
    #plt.plot(np.arange(1,max_L+1,1),poly(np.arange(1,max_L+1,1)),c='red',zorder=1) 
    if window:
        plt.plot(l_array[2:-2],np.convolve(neffect_array,kernel,mode = 'valid'),c='red',zorder=1) 
    # plt.plot(np.arange(1,max_L,3),max_pool,c='red',zorder=1) 
    if poly:
        xs = np.arange(1,max_L,1)
        neff_poly_model = olsres2.params[2]*np.power(xs,3) + olsres2.params[1]*np.power(xs,2) + olsres2.params[0]*xs
        plt.plot(xs,neff_poly_model,color='darkorange',zorder=10)
        plt.fill_between(xs,neff_poly_model,np.zeros(len(xs)),color='darkorange',alpha=0.2,zorder=1,label='Effective')
        plt.fill_between(xs,xs,neff_poly_model,color='grey',alpha=0.2,zorder=1,label='Ineffective')
        plt.legend(loc='upper left')
        
    plt.plot([0,n],[n,n],c='grey',zorder=3)
    plt.plot([n,n],[0,n],c='grey',zorder=3)

    if n > 10: 
        plt.annotate('No Spatial Correlation = ' + str(np.round(n,2)),[1.2,n+0.4],c='grey',zorder=3)
        if abs(n - neffective) > 1.5:
            plt.annotate('With Spatial Correlation = ' + str(np.round(neffective_model,2)),[1.2,neffective_model+0.4],c='darkorange',zorder=5)
            plt.plot([n,n],[0,neffective_model],c='darkorange',zorder=5); 
            plt.plot([0,n],[neffective_model,neffective_model],c='darkorange',zorder=5) 
    
    plt.ylim([1,max_L]); plt.xlim([1,max_L]); plt.xscale("linear"); plt.yscale("linear")
    plt.xlabel("Number of Samples"); plt.ylabel("Number of Independent Samples"); plt.title(r'Number of Independent Samples ($n_{effective}$) vs. Number of Samples')
    add_grid()
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.2, wspace=0.2, hspace=0.2); 
    plt.savefig('SpatialBootstrap.jpg',dpi=600,bbox_inches='tight')  
    plt.show()
    
interactive_plot1 = widgets.interactive_output(f_make1, {'n':n,'vrange':vrange,'seed':seed,'window':window,'poly':poly})
interactive_plot1.clear_output(wait = True)                # reduce flickering by delaying plot updating  

In [7]:
display(ui_all, interactive_plot1)                            # display the interactive plot

Output()

#### Spatial Bootstrap Demonstrations

I provide two methods for spatial bootstrap.

##### Spatial Bootstrap by Unconditional Simulation at Data Locations

This is the method proposed by Professor Deutsch with unconditional simulation at the data locations with a computationally efficient LUSIM approach, based on lower and upper decomposition of the covariance matrix.

* Under the assumption of stationary histogram, variogram
* simulation is conducted in Gaussian space, but may be transformed to other distributions

Here we visualize the spatial bootstrap, bootstrap accounting for data location and the spatial correlation via the variogram. 

* How many independent pieces of information do you have?

In [8]:
nreal = 100                                                   # number of realizations
tmean = 1000; tstdev = 200                                    # distribution mean and standard deviation
l = widgets.Text(value=r'                                       Spatial Bootstrap for Uncertainty in the Mean, Michael Pyrcz, Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

n = widgets.IntSlider(min=1, max = max_L, value = 1,step = 1,description = 'n',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
n.style.handle_color = 'gray'

vrange = widgets.FloatSlider(min=0.1, max = 200.0, value = 10.0,step=5,description = 'range',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
vrange.style.handle_color = 'gray'

ireal = widgets.IntSlider(min=1, max = nreal, value = 1,step = 1,description = 'l',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
ireal.style.handle_color = 'gray'

seed = widgets.IntSlider(min=100, max = 999, value = 1, description = 's',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
seed.style.handle_color = 'gray'

window = widgets.Checkbox(value=False,description='Conv. Fit',disabled=False,layout=Layout(width='200px', height='30px'))
poly = widgets.Checkbox(value=True,description='Poly. Fit',disabled=False,layout=Layout(width='200px', height='30px'))

ui1 = widgets.HBox([n,vrange,ireal,seed,],kwargs = {'justify_content':'center'})
ui2 = widgets.HBox([window,poly,],kwargs = {'justify_content':'center'})
ui_all2 = widgets.VBox([l,ui1,ui2],)

def f_make2(n,vrange,ireal,seed,window,poly):
    
    nreal = 100;
    maxf = 50; maxfboot = 15
    dx = -5; dy = 3; pdfy = 13.0
    
    np.random.seed(seed=seed)
    locations = np.random.rand(max_L*2)*100
    X,Y = np.split(locations,2)
    df = pd.DataFrame(np.vstack((X[:n],Y[:n])).T,columns = ['X','Y'])
    
    ax1 = plt.subplot(121)
    plt.scatter(df['X'],df['Y'],marker='x',s=30,color='red',edgecolor='black',lw=2,zorder=10,label = 'Spatial Data')
    plt.scatter(df['X'],df['Y'],marker='x',s=40,color='white',edgecolor='black',lw=4,zorder=9,label = 'Spatial Data')
    plt.gca().add_patch(plt.Rectangle((0, 0), 100, 100, fill=False,edgecolor='black',lw=2))
    
    xx = np.arange(-3,3,0.1)
    pdf = norm.pdf(xx, loc=0, scale=1) * pdfy
    px = np.linspace(0,10,len(xx))
    
    for ipts in range(0,n):
        if ipts == 0:
            circle1 = plt.Circle((df['X'][ipts],df['Y'][ipts]),vrange,fill=True,color='red',edgecolor=None,
                lw=0.0,alpha=0.05,zorder=1,label='Spatial Continuity')
        else:
            circle1 = plt.Circle((df['X'][ipts],df['Y'][ipts]),vrange,fill=True,color='red',edgecolor=None,
                lw=0.0,alpha=0.05,zorder=1)
        plt.gca().add_patch(circle1)
        
        plt.plot([df['X'][ipts]+dx,df['X'][ipts]+10+dx],[df['Y'][ipts]+dy,df['Y'][ipts]+dy],color='black')
        plt.plot([df['X'][ipts]+dx,df['X'][ipts]+dx],[df['Y'][ipts]+dy,df['Y'][ipts]+5+dy],color='black')
        plt.plot(px + df['X'][ipts] + dx,pdf + df['Y'][ipts] + dy,color = 'black',zorder=1)
    
    plt.xlim([-10,110]); plt.ylim([-10,110]); add_grid(); plt.xlabel("X (m)"); plt.ylabel("Y (m)") 
    plt.title('Random Spatial Dataset with Spatial Correlation Range Indicated')
    
    vario = GSLIB.make_variogram(nug=0.0,nst=1,it1=1,cc1=1.0,azi1=0.0,hmaj1=vrange,hmin1=vrange)
    neffective,realizations = n_effective(df,'X','Y',seed=seed,nreal=nreal,vario=vario)
    
    realizations = realizations*tstdev+tmean       # correct distribution from N[0,1] to target mean and st.dev.
    
    for ipts in range(0,n):
        y = df['Y'][ipts]+dy; x = df['X'][ipts]+dx+(realizations[ireal-1,ipts]-0.0)/2000.0*10
        plt.plot([x,x],[y,y+5],color='black',lw=3,zorder=1); plt.plot([x,x],[y,y+5],color='red',lw=1,zorder=3)
    
    ax2 = plt.subplot(122)
    plt.axis('off'); plt.xlim([0, 10]); plt.ylim([0, 10])
    
    oxp = 2.0; oyp = 6.7; xprange = 6.0; yprange = 3.0 # location of histogram of bootstrap realization
    xmin = 0.0; xmax = 2000.0; ymin = 0.0; ymax = maxfboot; nbin = 11
    
    plt.plot([oxp,oxp+xprange],[oyp,oyp],color='black'); plt.plot([oxp,oxp],[oyp,oyp+yprange],color='black')
    
    xpmin = oxp; xpmax = oxp+xprange; ypmin = oyp; ypmax = oyp+yprange
    xrange = xmax - xmin; yrange = ymax - ymin
    xhalf = (xmax-xmin)/(nbin-1)/2.0; xsize = xhalf*2.0
    xxhalf = xhalf*xprange/xrange; xxsize = xsize*xprange/xrange 

    xbins = np.linspace(xmin,xmax,nbin)
    ybins = np.linspace(ymin,maxfboot,11)
    xcents = np.linspace(xmin + xhalf,xmax-xhalf,nbin-1)
    
    for ibin, xbin in enumerate(xbins):
        xx = ((xbin-xmin)/xrange*xprange)+xpmin
        plt.plot([xx,xx],[ypmin,ypmax],color='grey',lw=0.2,zorder=1)
        if ibin % 2 == 0:
            plt.plot([xx,xx],[ypmin-yprange*0.1,ypmin],color='black',zorder=3)
            plt.annotate(np.round(xbin,1),(xx,ypmin-yprange*0.20),ha='center')
        else: 
            plt.plot([xx,xx],[ypmin-yprange*0.05,ypmin],color='black',zorder=3)
            plt.annotate(np.round(xbin,1),(xx,ypmin-yprange*0.13),ha='center')
    
    for ybin in ybins:
        #yy = (ybin)*3/maxfboot+6
        yy = ((ybin-ymin)/yrange*yprange)+ypmin
        plt.plot([xpmin-xprange*0.04,xpmin],[yy,yy],color='black',zorder=3)
        plt.plot([xpmin,xpmax],[yy,yy],color='grey',lw=0.2,zorder=1)
        plt.annotate(np.round(ybin,1),(xpmin-xprange*0.05,yy),ha='right')
    
    histboot = np.histogram(realizations[ireal-1,:], bins=xbins, weights=None)[0]
    average = np.average(realizations[ireal-1,:])
    
    for ibin, prop in enumerate(histboot):
        xx = ((xcents[ibin]-xmin)/xrange*xprange)+xpmin
        yy = histboot[ibin]*yprange/maxfboot
        ax2.add_patch(plt.Rectangle((xx-xxhalf,ypmin), xxsize, yy, lw=1, fc = 'darkorange',color='black', ))
        xavg = ((average-xmin)/xrange*xprange)+xpmin
        
    for ipts in range(0,n):
        xx = ((realizations[ireal-1,ipts]-xmin)/xrange*xprange)+xpmin 
        plt.scatter(xx,ypmin,color='red',edgecolor='black',s=20,alpha=1.0,zorder=100)
       
    plot_avg = True
    if plot_avg:
        xx = ((np.average(realizations[ireal-1,:])-xmin)/xrange*xprange)+xpmin
        plt.plot([xx,xx],[ypmin,ypmax],color='red',lw=2,ls='--')
        plt.annotate(r'$\overline{x}^{\ell} = $' + str(np.round(np.average(realizations[ireal-1,:]),2)),
                     (xx,ypmin+yprange*0.8),color='red',rotation=270,va='center')
        
    plt.annotate('Spatial Bootstrap Distribution Realization #' + str(ireal),(xpmin+xprange*0.5,ypmax+yprange*0.08),ha='center') 
    plt.annotate('Lithium Grade (ppm)',(xpmin+xprange*0.5,ypmin-yprange*0.27),ha='center') 
    plt.annotate('Frequency',(xpmin-xprange*0.18,ypmin+yprange*0.5),va='center',rotation = 90.0)   
    
    oxp = 2.0; oyp = 2.3; xprange = 6.0; yprange = 3.0 # location of histogram of averages
    xmin = 0.0; xmax = 2000.0; ymin = 0.0; ymax = maxfboot; nbin = 11
       
    xpmin = oxp; xpmax = oxp+xprange; ypmin = oyp; ypmax = oyp+yprange
    xrange = xmax - xmin; yrange = ymax - ymin
    xhalf = (xmax-xmin)/(nbin-1)/2.0; xsize = xhalf*2.0
    xxhalf = xhalf*xprange/xrange; xxsize = xsize*xprange/xrange
    xbins = np.linspace(xmin,xmax,nbin)
    ybins = np.linspace(ymin,maxfboot,11)
    xcents = np.linspace(xmin + xhalf,xmax-xhalf,nbin-1)
    xvalues = np.linspace(xmin,xmax,100)
    
    plt.plot([oxp,oxp+xprange],[oyp,oyp],color='black'); plt.plot([oxp,oxp],[oyp,oyp+yprange],color='black')
    
    for ibin, xbin in enumerate(xbins):
        xx = ((xbin-xmin)/xrange*xprange)+xpmin
        plt.plot([xx,xx],[ypmin,ypmax],color='grey',lw=0.2,zorder=1)
        if ibin % 2 == 0:
            plt.plot([xx,xx],[ypmin-yprange*0.1,ypmin],color='black',zorder=3)
            plt.annotate(np.round(xbin,1),(xx,ypmin-yprange*0.20),ha='center')
        else: 
            plt.plot([xx,xx],[ypmin-yprange*0.05,ypmin],color='black',zorder=3)
            plt.annotate(np.round(xbin,1),(xx,ypmin-yprange*0.13),ha='center')
                 
    for ybin in ybins:
        yy = ((ybin-ymin)/yrange*yprange)+ypmin
        plt.plot([xpmin-xprange*0.04,xpmin],[yy,yy],color='black',zorder=3)
        plt.plot([xpmin,xpmax],[yy,yy],color='grey',lw=0.2,zorder=1)
        plt.annotate(np.round(ybin,1),(xpmin-xprange*0.05,yy),ha='right')
        
    averages = np.average(realizations,axis=1)
    average = np.average(averages[:ireal])
    hist_avg = np.histogram(averages[:ireal], bins=xbins, weights=None)[0]
    
    for ibin, prop in enumerate(hist_avg):
        xx = ((xcents[ibin]-xmin)/xrange*xprange)+xpmin
        yy = hist_avg[ibin]*3/maxf
        ax2.add_patch(plt.Rectangle((xx-xxhalf,ypmin), xxsize, yy, lw=1, fc = 'gold',color='black',zorder=10))
        
    if ireal > 1:
        hist_avg_prior = np.histogram(averages[:ireal-1], bins=xbins, weights=None)[0]
        for ibin, prop in enumerate(hist_avg):
            yy = hist_avg_prior[ibin]*3/maxf
            xx = ((xcents[ibin]-xmin)/xrange*xprange)+xpmin
            ax2.add_patch(plt.Rectangle((xx-xxhalf,ypmin), xxsize, yy, lw=1, fc = 'darkorange',color='black',zorder=15))
        
    plt.annotate('Spatial Bootstrap Average Realizations',(xpmin+xprange*0.5,ypmax+yprange*0.08),ha='center') 
    plt.annotate('Bootstrap Average Lithium Grade (ppm)',(xpmin+xprange*0.5,ypmin-yprange*0.30),ha='center') 
    plt.annotate('Frequency',(xpmin-xprange*0.18,ypmin+yprange*0.5),va='center',rotation = 90.0) 
    
    mean = np.average(averages[:ireal]); stdev = np.std(averages[:ireal])
    P10 = np.percentile(averages[:ireal],10); P90 = np.percentile(averages[:ireal],90)
    plt.annotate('Spatial Bootstrap Uncertainty in Mean:',(0,0.8))
    plt.annotate('Mean: ' + str(np.round(mean,2)),(0,0.4),ha='left')
    plt.annotate('St.Dev.: ' + str(np.round(stdev,2)),(2.0,0.4),ha='left')
    plt.annotate('P10: ' + str(np.round(P10,2)),(0.0,0.0),ha='left')
    plt.annotate('P90: ' + str(np.round(P90,2)),(2.0,0.0),ha='left')
    
    if ireal > 1:
        xxmean = ((mean-xmin)/xrange*xprange)+xpmin
        plt.plot([xxmean,xxmean],[ypmin,ypmax],color='red',lw=2.0,zorder=100)
        plt.annotate('Mean: ' + str(np.round(mean,2)),(xxmean,ypmin+yprange*0.9),rotation=270.0,color='red',va='top')
        
        xxP10 = ((P10-xmin)/xrange*xprange)+xpmin
        plt.plot([xxP10,xxP10],[ypmin,ypmax],color='red',lw=2.0,ls='--',zorder=100)
        plt.annotate('P10: ' + str(np.round(P10,2)),(xxP10,ypmin+yprange*0.9),rotation=270.0,color='red',va='top')
        
        xxP90 = ((P90-xmin)/xrange*xprange)+xpmin
        plt.plot([xxP90,xxP90],[ypmin,ypmax],color='red',lw=2.0,ls='--',zorder=100)
        plt.annotate('P90: ' + str(np.round(P90,2)),(xxP90,ypmin+yprange*0.9),rotation=270.0,color='red',va='top')
       
    if ireal > 5:
        PDFModel = sm.nonparametric.KDEUnivariate(averages[:ireal]).fit()
        yPDF = PDFModel.evaluate(xvalues)*len(averages[:ireal])*xsize/3.0
        xxvalues = ((xvalues-xmin)/xrange*xprange)+xpmin; yyPDF = ((yPDF-ymin)/yrange*yprange)+ypmin
        plt.plot(xxvalues,yyPDF,color='black',zorder=200)
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.2, wspace=0.2, hspace=0.2)
    plt.savefig('SpatialBootstrap.jpg',dpi=600,bbox_inches='tight')  
    plt.show()
    
interactive_plot2 = widgets.interactive_output(f_make2, {'n':n,'vrange':vrange,'ireal':ireal,'seed':seed,'window':window,'poly':poly})
interactive_plot2.clear_output(wait = True)                # reduce flickering by delaying plot updating     

In [9]:
display(ui_all2, interactive_plot2)                            # display the interactive plot

Output()

##### Spatial Bootstrap By Sampling from Images

This is the method proposed by Prof. Journel (SCRF).

* this method is general and does not require a specific spatial continuity model nor distribution assumption

* we assume stationarity of the specific spatial patterns, as we use sample translation to access replicates

###### Make a Stochastic Image

We use the convolutional approach to calculating a unconditional sequential Gaussian simulation by:
    
1. Calculate a set of random Gaussian values over the area of interest. Note, we add padding equal to half the kernel size to avoid edge effects.

2. Convolution with a anisotropic, unbiased kernel

In [10]:
tmean = 1000.0; tstdev = 200.0; vmin = 400.0; vmax = 1600.0   # lithium grade distribution (ppm)
nx = 50; ny = 50; xmin = 0; xmax = 1000; ymin = 0.0; ymax = 1000.0 # specify the grid

xrange = xmax - xmin; yrange = ymax - ymin
xsiz = xrange/nx; ysiz = yrange/ny
xmn = xmin + xsiz/2.0; ymn = ymin + ysiz/2.0; 

npadding = 30; nky = 31; nkx = 31; cky = nky/2; ckx = nky/2; vrangex = 150; vrangey = 50 #specify the kernel

kernel = np.zeros((nky,nkx)); geo_dist = np.zeros((nky,nkx))  # make the kernel
for iy in range(0,nky):
    for ix in range(0,nkx):
        geo_dist[iy,ix] = 1/(((ix-ckx)*xsiz/vrangex)**2 + ((iy-cky)*ysiz/vrangey)**2 + 0.1)           

kernel[geo_dist > 1.0] = 1.0
kernel = kernel / np.sum(kernel.flatten())                    # unbiasedness, sum to 1.0

pfield = np.random.normal(size=(ny+npadding*2,nx+npadding*2))
pfield = signal.fftconvolve(pfield,kernel,mode='same')        # convolution

pfield = np.reshape(pfield[npadding:-npadding,npadding:-npadding],(ny,nx))
pfield = GSLIB.affine(pfield,tmean,tstdev)

###### Dashboard with Spatial Bootstrap from an Image / Model

Here's the dashboard for spatial bootstrap from an image.

In [13]:
import matplotlib as mpl

nreal = 100                                                   # number of realizations
l = widgets.Text(value=r'                Spatial Bootstrap, Resampling from Images or Models for Uncertainty in the Mean, Michael Pyrcz, Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

n = widgets.IntSlider(min=1, max = max_L, value = 1,step = 1,description = 'n',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
n.style.handle_color = 'gray'

ireal = widgets.IntSlider(min=1, max = nreal, value = 1,step = 1,description = 'l',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
ireal.style.handle_color = 'gray'

seed = widgets.IntSlider(min=100, max = 999, value = 1, description = 's',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
seed.style.handle_color = 'gray'

ui5 = widgets.HBox([n,ireal,seed,],kwargs = {'justify_content':'center'})
ui_all3 = widgets.VBox([l,ui5],)

def f_make3(n,ireal,seed,):
  
    maxf = 50; maxfboot = 15                                      # histogram maximum frequncies on the plots
    dx = -5; dy = 3; pdfy = 13.0
    template = 200.0; tsize = 5.0                                 # data template size and minimum font size on plots
    
    np.random.seed(seed=seed)
    values = np.random.rand(max_L*2)*template-template/2.0
    X_temp,Y_temp = np.split(values,2)
    dftemp = pd.DataFrame(np.vstack((X_temp[:n],Y_temp[:n])).T,columns = ['X','Y'])
    
    cmap = matplotlib.cm.get_cmap('inferno')
    
    ax1 = plt.subplot(111)
    plt.axis('off')
    
    plt.xlim([0, 22]); plt.ylim([0, 13])
    
    oxp = 2.0; oyp = 2.0; xprange = 10.0; yprange = 10.0          # location of histogram of averages
    
    nx = 50; ny = 50; xmin = 0; xmax = 1000; ymin = 0.0; ymax = 1000.0 # specify the grid
    xrange = xmax - xmin; yrange = ymax - ymin
    xsiz = xrange/nx; ysiz = yrange/ny
    xmn = xmin + xsiz/2.0; ymn = ymin + ysiz/2.0; 
    xpmin = oxp; xpmax = oxp+xprange; ypmin = oyp; ypmax = oyp+yprange
    xrange = xmax - xmin; yrange = ymax - ymin
    xpsiz = xsiz*xprange/xrange; ypsiz = ysiz*yprange/yrange
    
    xcents = np.linspace(xmn,xmax-xsiz/2.0,nx)
    ycents = np.linspace(ymax-ysiz/2.0,ymn,ny)
    values = np.linspace(vmin,vmax,100)
    
    plt.plot([xpmin,xpmin,xpmax,xpmax,xpmin],[ypmin,ypmax,ypmax,ypmin,ypmin],color='black',lw = 1.0,zorder=200)
    
    gap = 0.1                                                     # pixelplot of image
    for iy in range(0,ny):
        ylowcorner = ycents[iy] - ysiz/2.0
        yylowcorner = ((ylowcorner-ymin)/yrange*yprange)+ypmin + ypsiz*gap
        for ix in range(0,nx):
            #x = xmn + ix * xsiz
            xlowcorner = xcents[ix] - xsiz/2.0
            #xx = ((x-xmin)/xrange*xprange)+xpmin
            xxlowcorner = ((xlowcorner-xmin)/xrange*xprange)+xpmin + xpsiz*gap
            value = pfield[iy,ix]
            color = (value - vmin)/(vmax-vmin)
            ax1.add_patch(plt.Rectangle((xxlowcorner,yylowcorner), xpsiz*(1-2*gap), ypsiz*(1-2*gap), lw=0.1, fc = plt.cm.inferno(color),color='black',
                                        zorder=10))
    
    nbin = 51                                                     # x axis
    xbins = np.linspace(xmin,xmax,nbin)
    for ibin, xbin in enumerate(xbins):
        xx = ((xbin-xmin)/xrange*xprange)+xpmin
        plt.plot([xx,xx],[ypmin,ypmax],color='grey',lw=0.2,zorder=1)
        if ibin % 5 == 0:
            plt.plot([xx,xx],[ypmin-yprange*0.02,ypmin],color='black',lw=1.0,zorder=3)
            plt.annotate(int(np.round(xbin,0)),(xx,ypmin-yprange*0.05),ha='center',size=tsize)
        else: 
            plt.plot([xx,xx],[ypmin-yprange*0.01,ypmin],color='black',lw=1.0,zorder=3)
    
    ybins = np.linspace(ymin,ymax,nbin)                           # y axis
    for ibin, ybin in enumerate(ybins):
        yy = ((ybin-ymin)/yrange*yprange)+ypmin
        plt.plot([xpmin,xpmax],[yy,yy],color='grey',lw=0.2,zorder=1)
        if ibin % 5 == 0:
            plt.plot([xpmin-xprange*0.02,xpmin],[yy,yy],color='black',lw=1.0,zorder=3)
            plt.annotate(int(np.round(ybin,0)),(xpmin-xprange*0.05,yy),va='center',rotation=90.0,size=tsize)
        else: 
            plt.plot([ypmin-yprange*0.01,ypmin],[yy,yy],color='black',lw=1.0,zorder=3)
    
    realizations = np.zeros((nreal,n)); xreals = np.zeros((nreal,n)); yreals = np.zeros((nreal,n))
    np.random.seed(seed=seed)
    dyo = np.random.rand(nreal)*(yrange - template) + template*0.5
    dxo = np.random.rand(nreal)*(xrange - template) + template*0.5
    dyyo = ((dyo-ymin)/yrange*yprange)+ypmin
    dxxo = ((dxo-xmin)/xrange*xprange)+xpmin
    
    for i in range(0,nreal):                                      # get all realizations to ensure repeatability with seed
        dftemp_real = dftemp.copy(deep = True)
        dftemp_real['Y'] = dftemp_real['Y'] + dyo[i]
        dftemp_real['X'] = dftemp_real['X'] + dxo[i]
        dfspboot = GSLIB.sample(pfield, xmin, ymin, xsiz, 'Value',dftemp_real,'X','Y')
        realizations[i,:] = dfspboot['Value']
        xreals[i,:] = dftemp_real['X']; yreals[i,:] = dftemp_real['Y']
        
    dymin = dyo - template/2.0; dymax = dymin + template
    dxmin = dxo - template/2.0; dxmax = dxmin + template
    
    dyymin = ((dymin-ymin)/yrange*yprange)+ypmin; dxxmin = ((dxmin-xmin)/xrange*xprange)+xpmin
    dyymax = ((dymax-ymin)/yrange*yprange)+ypmin; dxxmax = ((dxmax-xmin)/xrange*xprange)+xpmin
    
    plt.plot([dxxmin[ireal-1],dxxmin[ireal-1],dxxmax[ireal-1],dxxmax[ireal-1],dxxmin[ireal-1]], # plot data template extents
             [dyymin[ireal-1],dyymax[ireal-1],dyymax[ireal-1],dyymin[ireal-1],dyymin[ireal-1]],color='black',lw = 1.0,zorder=200)
    plt.plot([dxxmin[ireal-1],dxxmin[ireal-1],dxxmax[ireal-1],dxxmax[ireal-1],dxxmin[ireal-1]], # plot data template extents
             [dyymin[ireal-1],dyymax[ireal-1],dyymax[ireal-1],dyymin[ireal-1],dyymin[ireal-1]],color='white',lw = 3.0,zorder=190)
    plt.scatter(dxxo[ireal-1],dyyo[ireal-1],marker='x',color='black',s=40,zorder=200)
    
    yy = ((yreals[ireal-1,:]-ymin)/yrange*yprange)+ypmin
    xx = ((xreals[ireal-1,:]-xmin)/xrange*xprange)+xpmin
    vals = (realizations[ireal-1,:] - vmin)/(vmax - vmin)
    
    plt.scatter(xx,yy,c=vals,edgecolor=None,marker='o',lw=1,s=15,alpha=1.0,
                vmin=0.0,vmax=1.0,cmap=cmap,zorder=300) 
    plt.scatter(xx,yy,color='black',edgecolor='white',marker='o',lw=3,s=22,alpha=1.0,zorder=299)
    plt.scatter(xx,yy,color='white',edgecolor='black',marker='o',lw=5,s=23,alpha=1.0,zorder=298)        
    
    plt.annotate('X (m)',(xpmin + xprange*0.5,ypmin - yprange*0.1),ha = 'center',size=tsize*1.6)
    plt.annotate('Y (m)',(xpmin - xprange*0.1,ypmin + yprange*0.5),rotation = 90.0,va = 'center',size=tsize*1.6)
    plt.annotate('Lithium Model with Spatial Bootstrap Realization',(xpmin + xprange*0.5,ypmax + yprange*0.02),ha = 'center',size=tsize*1.6)
    
    #### Bootstrap Histogram
    
    oxp = 15.0; oyp = 8.0; xprange = 6.0; yprange = 3.0           # location of histogram of bootstrap realization
    nbin = 21; xmin = 0; xmax = 1000; ymin = 0.0; ymax = 15.0     # specify the grid
    xrange = xmax - xmin; yrange = ymax - ymin
    
    plt.plot([oxp,oxp+xprange],[oyp,oyp],color='black'); plt.plot([oxp,oxp],[oyp,oyp+yprange],color='black')
        
    xpmin = oxp; xpmax = oxp+xprange; ypmin = oyp; ypmax = oyp+yprange
    vhalf = (vmax-vmin)/(nbin-1)/2.0; vsize = vhalf*2.0
    vrange = vmax - vmin; yrange = ymax - ymin
    vvhalf = vhalf*xprange/vrange; vvsize = vsize*xprange/vrange 

    vbins = np.linspace(vmin,vmax,nbin)
    ybins = np.linspace(ymin,maxfboot,11)
    vcents = np.linspace(vmin + vhalf,vmax-vhalf,nbin-1)
    
    for ibin, vbin in enumerate(vbins):
        vv = ((vbin-vmin)/vrange*xprange)+xpmin
        plt.plot([vv,vv],[ypmin,ypmax],color='grey',lw=0.2,zorder=1)
        if ibin % 2 == 0:
            plt.plot([vv,vv],[ypmin-yprange*0.1,ypmin],lw=0.4,color='black',zorder=3)
            plt.annotate(int(np.round(vbin,0)),(vv,ypmin-yprange*0.23),ha='center',size=tsize)
        else: 
            plt.plot([vv,vv],[ypmin-yprange*0.05,ypmin],lw=0.4,color='black',zorder=3)
            plt.annotate(int(np.round(vbin,0)),(vv,ypmin-yprange*0.13),ha='center',size=tsize)   
                 
    for ybin in ybins:
        yy = ((ybin-ymin)/yrange*yprange)+ypmin
        plt.plot([xpmin-xprange*0.02,xpmin],[yy,yy],color='black',lw=0.4,zorder=3)
        plt.plot([xpmin,xpmax],[yy,yy],color='grey',lw=0.4,zorder=1)
        plt.annotate(int(np.round(ybin,0)),(xpmin-xprange*0.05,yy),ha='right',size=tsize)
    
    histboot = np.histogram(realizations[ireal-1,:], bins=vbins, weights=None)[0]
    
    average = np.average(realizations[ireal-1,:])
    
    for ibin, prop in enumerate(histboot):
        vv = ((vcents[ibin]-vmin)/vrange*xprange)+xpmin
    #     print(vv)
        yy = histboot[ibin]*yprange/maxfboot
    #     print(yy)
        ax1.add_patch(plt.Rectangle((vv-vvhalf,ypmin), vvsize, yy, lw=1, fc = 'darkorange',color='black', ))
        vavg = ((average-vmin)/vrange*xprange)+xpmin
        
    for ipts in range(0,n):
        vv = ((realizations[ireal-1,ipts]-vmin)/vrange*xprange)+xpmin 
        plt.scatter(vv,ypmin,color='red',edgecolor='black',s=20,alpha=1.0,zorder=100)
        
    plot_avg = True
    if plot_avg:
        vv = ((np.average(realizations[ireal-1,:])-vmin)/vrange*xprange)+xpmin
        plt.plot([vv,vv],[ypmin,ypmax],color='red',lw=2,ls='--')
        plt.annotate(r'$\overline{x}^{\ell} = $' + str(int(np.round(np.average(realizations[ireal-1,:]),0))),
                     (vv,ypmin+yprange*0.6),color='red',rotation=270,size=tsize*1.2)
        
    plt.annotate('Spatial Bootstrap Distribution Realization #' + str(ireal),(xpmin+xprange*0.5,ypmax+yprange*0.08),ha='center',size=tsize*1.6) 
    plt.annotate('Lithium Grade (ppm)',(xpmin+xprange*0.5,ypmin-yprange*0.4),ha='center',size=tsize*1.6) 
    plt.annotate('Frequency',(xpmin-xprange*0.14,ypmin+yprange*0.5),va='center',rotation = 90.0,size=tsize*1.6)   
    
    oxp = 15.0; oyp = 3.0; xprange = 6.0; yprange = 3.0           # location of histogram of averages
    nbin = 21; ymin = 0.0; ymax = 15.0
    xpmin = oxp; xpmax = oxp+xprange; ypmin = oyp; ypmax = oyp+yprange
    vhalf = (vmax-vmin)/(nbin-1)/2.0; vsize = vhalf*2.0
    vvhalf = vhalf*xprange/vrange; vvsize = vsize*xprange/vrange 
    vrange = vmax - vmin; yrange = ymax - ymin
    vbins = np.linspace(vmin,vmax,nbin)
    ybins = np.linspace(ymin,maxfboot,11)
    vcents = np.linspace(vmin + vhalf,vmax-vhalf,nbin-1)
    
    plt.plot([oxp,oxp+xprange],[oyp,oyp],color='black'); plt.plot([oxp,oxp],[oyp,oyp+yprange],color='black')
    
    for ibin, vbin in enumerate(vbins):
        vv = ((vbin-vmin)/vrange*xprange)+xpmin
        plt.plot([vv,vv],[ypmin,ypmax],color='grey',lw=0.2,zorder=1)
        if ibin % 2 == 0:
            plt.plot([vv,vv],[ypmin-yprange*0.1,ypmin],lw=0.4,color='black',zorder=3)
            plt.annotate(int(np.round(vbin,0)),(vv,ypmin-yprange*0.23),ha='center',size=tsize)
        else: 
            plt.plot([vv,vv],[ypmin-yprange*0.05,ypmin],lw=0.4,color='black',zorder=3)
            plt.annotate(int(np.round(vbin,0)),(vv,ypmin-yprange*0.13),ha='center',size=tsize)   
                 
    for ybin in ybins:
        yy = ((ybin-ymin)/yrange*yprange)+ypmin
        plt.plot([xpmin-xprange*0.02,xpmin],[yy,yy],color='black',lw=0.4,zorder=3)
        plt.plot([xpmin,xpmax],[yy,yy],color='grey',lw=0.4,zorder=1)
        plt.annotate(int(np.round(ybin,0)),(xpmin-xprange*0.05,yy),ha='right',size=tsize)
        
    averages = np.average(realizations,axis=1)
    # print(realizations)
    average = np.average(averages[:ireal])
    hist_avg = np.histogram(averages[:ireal], bins=vbins, weights=None)[0]
    for ibin, prop in enumerate(histboot):
        vv = ((vcents[ibin]-vmin)/vrange*xprange)+xpmin
        yy = hist_avg[ibin]*yprange/maxfboot
        ax1.add_patch(plt.Rectangle((vv-vvhalf,ypmin), vvsize, yy, lw=1, fc = 'gold',color='black',zorder=20))
        vavg = ((average-vmin)/vrange*xprange)+xpmin

    if ireal > 1:
        hist_avg = np.histogram(averages[:ireal-1], bins=vbins, weights=None)[0]
        for ibin, prop in enumerate(histboot):
            vv = ((vcents[ibin]-vmin)/vrange*xprange)+xpmin
            yy = hist_avg[ibin]*yprange/maxfboot
            ax1.add_patch(plt.Rectangle((vv-vvhalf,ypmin), vvsize, yy, lw=1, fc = 'darkorange',color='black',zorder=25))
            vavg = ((average-vmin)/vrange*xprange)+xpmin
        
    plt.annotate('Spatial Bootstrap Average Realizations',(xpmin+xprange*0.5,ypmax+yprange*0.08),ha='center',size=tsize*1.6) 
    plt.annotate('Bootstrap Average Lithium Grade (ppm)',(xpmin+xprange*0.5,ypmin-yprange*0.40),ha='center',size=tsize*1.6) 
    plt.annotate('Frequency',(xpmin-xprange*0.14,ypmin+yprange*0.5),va='center',rotation = 90.0,size=tsize*1.6) 
    
    mean = np.average(averages[:ireal]); stdev = np.std(averages[:ireal])
    P10 = np.percentile(averages[:ireal],10); P90 = np.percentile(averages[:ireal],90)
    plt.annotate('Spatial Bootstrap Uncertainty in Mean:',(0,0.8),size=tsize*1.6,zorder=400)
    plt.annotate('Mean: ' + str(np.round(mean,2)),(0,0.4),ha='left',size=tsize*1.4,zorder=400)
    plt.annotate('St.Dev.: ' + str(np.round(stdev,2)),(2.0,0.4),ha='left',size=tsize*1.4,zorder=400)
    plt.annotate('P10: ' + str(np.round(P10,2)),(0.0,0.0),ha='left',size=tsize*1.4,zorder=400)
    plt.annotate('P90: ' + str(np.round(P90,2)),(2.0,0.0),ha='left',size=tsize*1.4,zorder=400)
    
    if ireal > 1:
        vvmean = ((mean-vmin)/vrange*xprange)+xpmin
        plt.plot([vvmean,vvmean],[ypmin,ypmax],color='red',lw=2.0,zorder=200)
        plt.annotate('Mean: ' + str(np.round(mean,2)),(vvmean,ypmin+yprange*0.9),rotation=270.0,color='red',va='top',size=tsize*1.3,zorder=200)
        
        vvP10 = ((P10-vmin)/vrange*xprange)+xpmin
        plt.plot([vvP10,vvP10],[ypmin,ypmax],color='red',lw=2.0,ls='--',zorder=200)
        plt.annotate('P10: ' + str(np.round(P10,2)),(vvP10,ypmin+yprange*0.9),rotation=270.0,color='red',va='top',size=tsize*1.3,zorder=200)
        
        vvP90 = ((P90-vmin)/vrange*xprange)+xpmin
        plt.plot([vvP90,vvP90],[ypmin,ypmax],color='red',lw=2.0,ls='--',zorder=200)
        plt.annotate('P90: ' + str(np.round(P90,2)),(vvP90,ypmin+yprange*0.9),rotation=270.0,color='red',va='top',size=tsize*1.3,zorder=200)
    
    if ireal > 5:
        PDFModel = sm.nonparametric.KDEUnivariate(averages[:ireal]).fit()
        yPDF = PDFModel.evaluate(values)*len(averages[:ireal])*vsize
        vvalues = ((values-vmin)/vrange*xprange)+xpmin; yyPDF = ((yPDF-ymin)/yrange*yprange)+ypmin
        plt.plot(vvalues,yyPDF,color='black',zorder=100)

    fraction = 0.01
    norm = mpl.colors.Normalize(vmin=vmin,vmax=vmax)
    cbar = ax1.figure.colorbar(
            mpl.cm.ScalarMappable(norm=norm, cmap='inferno'),
            ax=ax1, pad=-0.78, extend='both', fraction=fraction)

    ticklabs = cbar.ax.get_yticklabels()
    cbar.ax.set_yticklabels(ticklabs, fontsize=6)
    
    plt.annotate('Lithium Grade (ppm)',(13.4,6.5),rotation=270.0,va='center',size=tsize*1.2)
        
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=0.7, top=1.0, wspace=0.2, hspace=0.2)
    #plt.savefig('SpatialBootstrap.jpg',dpi=600,bbox_inches='tight')  
    plt.show()
      
interactive_plot3 = widgets.interactive_output(f_make3, {'n':n,'ireal':ireal,'seed':seed})
interactive_plot3.clear_output(wait = True)                # reduce flickering by delaying plot updating     

In [14]:
display(ui_all3, interactive_plot3)                            # display the interactive plot

Output()

#### Comments

This was an interactive demonstration spatial bootstrap.
  
I hope this was helpful,

*Michael*

#### The Author:

### Michael Pyrcz, Professor, The University of Texas at Austin 
*Novel Data Analytics, Geostatistics and Machine Learning Subsurface Solutions*

With over 17 years of experience in subsurface consulting, research and development, Michael has returned to academia driven by his passion for teaching and enthusiasm for enhancing engineers' and geoscientists' impact in subsurface resource development. 

For more about Michael check out these links:

#### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

#### Want to Work Together?

I hope this content is helpful to those that want to learn more about subsurface modeling, data analytics and machine learning. Students and working professionals are welcome to participate.

* Want to invite me to visit your company for training, mentoring, project review, workflow design and / or consulting? I'd be happy to drop by and work with you! 

* Interested in partnering, supporting my graduate student research or my Subsurface Data Analytics and Machine Learning consortium (co-PIs including Profs. Foster, Torres-Verdin and van Oort)? My research combines data analytics, stochastic modeling and machine learning theory with practice to develop novel methods and workflows to add value. We are solving challenging subsurface problems!

* I can be reached at mpyrcz@austin.utexas.edu.

I'm always happy to discuss,

*Michael*

Michael Pyrcz, Ph.D., P.Eng. Professor, Cockrell School of Engineering and The Jackson School of Geosciences, The University of Texas at Austin

#### More Resources Available at: [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)